In [ ]:
import tensorflow as tf
import numpy as np
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus: 
    try: # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus: 
            tf.config.experimental.set_memory_growth(gpu, True)
            
        logical_gpus = tf.config.experimental.list_logical_devices('GPU') 
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
    
    except RuntimeError as e: # Memory growth must be set before GPUs have been initialized 
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
def performance_check(channel=1):
    
    aucs = []
    accs = []
    recalls = []
    specs = []
    f1scores = []

    x_valid = np.load('../data/%dc/x_valid.npy' % (channel))
    y_valid = np.load('../data/%dc/y_valid.npy' % (channel))

    if channel==1:
        x_valid = x_valid.reshape(x_valid.shape[0], 64, 64, 1)

    y_valid = y_valid.astype(np.uint8)
    x_val_nor = (x_valid / 255.)

    model = tf.keras.models.load_model('mc/%dc/ori-loss-model.h5' % (channel))
    print(x_val_nor.shape)

    pred = model.predict(x_val_nor)
    
    auc_score = tf.keras.metrics.AUC()
    auc_score.update_state(y_valid, pred)
    aucs.append(auc_score.result().numpy())

    for index, i in enumerate(pred):
        if i >= 0.5:
            pred[index] = 1
        else:
            pred[index] = 0

    acc = tf.keras.metrics.Accuracy()
    acc.update_state(y_valid, pred)
    accs.append(acc.result().numpy())

    recall = tf.keras.metrics.Recall()
    recall.update_state(y_valid, pred)
    recalls.append(recall.result().numpy())

    spec = tf.keras.metrics.SpecificityAtSensitivity(recall.result().numpy())
    spec.update_state(y_valid, pred)
    specs.append(spec.result().numpy())

    precision = tf.keras.metrics.Precision()
    precision.update_state(y_valid, pred)

    f1score = 2 * recall.result().numpy() * precision.result().numpy() / (precision.result().numpy() + recall.result().numpy())
    f1scores.append(f1score)
    
    return accs, aucs, recalls, specs, f1scores

In [ ]:
accs, aucs, recalls, specs, f1scores = performance_check(channel=1)
accs_2c, aucs_2c, recalls_2c, specs_2c, f1scores_2c = performance_check(channel=2)

(592, 64, 64, 1)
(564, 64, 64, 2)


In [ ]:
print(accs)

[0.8496622]


In [ ]:
print(accs_2c)

[0.8900709]


## model별로 성능을 비교한 그래프 그려보기
## matplotlib, seaborn 등 라이브러리 이용